In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import astropy.io.fits as pyfits
import astropy.utils as autils
import requests
import json
import datetime
from pprint import pprint as pp
import os
import astropy.io.fits as fits
import astropy.io.ascii as ascii
import copy

# Jielai added modules f
import subprocess
import seaborn as sns

# Time Counter function
import time
def tic():
    tic.start = time.perf_counter()
def toc():
    elapsed_seconds = time.perf_counter() - tic.start
    return elapsed_seconds # fractional

# NOAO server Settings
natroot = 'https://astroarchive.noirlab.edu'
assert natroot == 'https://astroarchive.noirlab.edu', 'Notebook does NOT point to PRODUCTION'
print(f"Using server on {natroot}")
adsurl = f'{natroot}/api/adv_search'
print(f"adsurl = {adsurl}")
apiurl = f'{adsurl}/fasearch/?limit=200000'
print(f'Using API url: {apiurl}')

# Start the timer
print(f'Started on: {str(datetime.datetime.now())}')
tic() # Start timing the run of this notebook

Using server on https://astroarchive.noirlab.edu
adsurl = https://astroarchive.noirlab.edu/api/adv_search
Using API url: https://astroarchive.noirlab.edu/api/adv_search/fasearch/?limit=200000
Started on: 2022-02-19 14:25:00.589799


In [2]:
def  get_radec_maxmin(RAcentre,DECcentre,search_radius_deg, debug=False):

    dec_min = DECcentre - search_radius_deg
    dec_max = DECcentre + search_radius_deg
    if dec_min<-90.0: dec_min=-90.0
    if dec_max>90.0: dec_max=90.0
    if dec_min==-90.0 or dec_max==90.0:
        ra_min = 0
        ra_max = 360.0
    else:
        costerm = min(np.cos(dec_min*np.pi/180.0),np.cos(dec_max*np.pi/180.0))
        ra_min = RAcentre-search_radius_deg*1./costerm
        ra_max = RAcentre+search_radius_deg*1./costerm
        if ra_min<0: ra_min+=360.0
        if ra_max>360.0: ra_max-=360.0

    if debug:
        print('**** DEBUG: ',dec_min, dec_max)
        print('**** DEBUG: ',ra_min, ra_max)

    return ra_min,ra_max,dec_min,dec_max
def makedirs_(out_path):
    out_dir = '/'.join(out_path.split('/')[0:-1])
    os.makedirs(out_dir, exist_ok=True)
    return None

In [12]:
pw = 'XXX'
usrname = 'XXX'
search_radius_deg = 0.5
caldat1g = '2022-02-17'
caldat2g = '2022-02-23'
raw_dir = '/Users/jielaizhang/Desktop/' # where downloaded stuff is saved
RA  = 203.6800149
DEC = 33.2167887

In [13]:
ra_min,ra_max,dec_min,dec_max = get_radec_maxmin(RA,DEC,search_radius_deg,debug=False)
if ra_min>ra_max:
    raise RuntimeError('This needs to be fixed!!')
    # Need to perform two searches to account for overlap

# Json needed to make a request to NOAO archive. 
jj = {    
        "outfields" : [
            "md5sum","release_date","proposal","archive_filename","original_filename","url",
            "proc_type","prod_type","ifilter","exposure",
            "ra_center","dec_center","dateobs_min","dateobs_max","caldat","filesize",
            "seeing"
            ],
        "search" : [ ["instrument", 'decam'],["telescope", 'ct4m'],["obs_type", 'object'],
                     ['ra_center',ra_min,ra_max],['dec_center',dec_min,dec_max],
                     ["caldat",caldat1g,caldat2g],
                     ["proc_type","instcal","stacked"],['prod_type','image']
                   ]
    }
ads_df = pd.DataFrame(requests.post(apiurl,json=jj).json()[1:])
print('Retrieved: ',len(ads_df))
if len(ads_df) != 0:
    print(np.unique(ads_df['caldat']))
    print(np.unique(ads_df['exposure']))

Retrieved:  15
['2022-02-17']
[ 50.  60.  70. 100. 120. 140.]


In [14]:
pd.set_option('display.max_colwidth', -1)
ads_df[['archive_filename','proc_type','exposure','caldat','seeing','url']]

,archive_filename,proc_type,exposure,caldat,seeing,url
0,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085230_osi_g_kntrap.fits.fz,stacked,120.0,2022-02-17,NaN,https://astroarchive.noirlab.edu/api/retrieve/0539c38c4b87cb80fa19f130f85e7f4a/
1,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085527_osi_i_kntrap.fits.fz,stacked,100.0,2022-02-17,NaN,https://astroarchive.noirlab.edu/api/retrieve/0ee03d9a99a01428b2624169234be9af/
2,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085806_ooi_z_v1.fits.fz,instcal,70.0,2022-02-17,1.18,https://astroarchive.noirlab.edu/api/retrieve/2c0dc45a3d269a1f960dd172122deebd/
3,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085527_ooi_i_kntrap.fits.fz,instcal,50.0,2022-02-17,1.30,https://astroarchive.noirlab.edu/api/retrieve/4da9b738e0d484135211416a34fff512/
4,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085945_ooi_z_kntrap.fits.fz,instcal,70.0,2022-02-17,1.37,https://astroarchive.noirlab.edu/api/retrieve/672d072f66a3f31648c1bd4f950e4406/
5,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085806_osi_z_kntrap.fits.fz,stacked,140.0,2022-02-17,NaN,https://astroarchive.noirlab.edu/api/retrieve/6a86f73b994c837e746cb264917d622a/
6,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085527_ooi_i_v1.fits.fz,instcal,50.0,2022-02-17,1.30,https://astroarchive.noirlab.edu/api/retrieve/75cc2a4066e28e7b40abefb2011d328e/
7,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085945_ooi_z_v1.fits.fz,instcal,70.0,2022-02-17,1.37,https://astroarchive.noirlab.edu/api/retrieve/76c8cf081fba811804f25a1be3e3d9c7/
8,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085359_ooi_g_v1.fits.fz,instcal,60.0,2022-02-17,1.54,https://astroarchive.noirlab.edu/api/retrieve/913599dcb732c3ad1a3d74d9549adafd/
9,/net/archive/pipe/20220217/ct4m/2022A-679480/c4d_220218_085230_ooi_g_v1.fits.fz,instcal,60.0,2022-02-17,1.62,https://astroarchive.noirlab.edu/api/retrieve/9c083bc08a4bfed1053534980770b49b/
